In [ ]:
# want to use the profile package, need to upgrade first
!pip install pandas-profiling -U

In [ ]:
import pandas as pd

# Instagram user profile details
user_profile = pd.read_csv('https://se-data-test.s3.ca-central-1.amazonaws.com/user_details_v1.csv')

In [ ]:
# use the pandas profiling package to explore further
from pandas_profiling import ProfileReport

profile = ProfileReport(user_profile)
profile

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# grab some of the relevant columns
user_profile_clean = user_profile[['input.user_id', # match in comment dataset
                                   'user.username', # ig handle
                                   'user.biography', # services offered
                                   'user.external_url', # website
                                   'user.address_street', # city 
                                   'user.full_name', # pro name
                                   'user.category', # business type
                                   'user.city_name', # city
                                   'user.contact_phone_number', # phone number
                                   'user.public_email', # email
                                   'user.public_phone_country_code', # country
                                   'user.public_phone_number', # phone
                                   'user.zip', # zip postal
                                   'user.is_business', # business pro
                                   'user.is_potential_business', # business
                                   'user.account_type', # business pro?
                                   'user.latitude',
                                   'user.longitude'
                                   ]].copy()

In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

def country_from_coords(x):
    try:
        return geolocator.reverse(x)
    except:
        return 'error:' + str(x)

geolocator = Nominatim(user_agent="a")
user_profile_clean["Country"] = user_profile_clean.apply(lambda x : country_from_coords(str(x["user.latitude"])+", "+str(x["user.longitude"])), axis=1)


/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py:256: UserWarning: Unable to parse the string as Point: "Failed to create Point instance from string: unknown format.". Using the value as-is for the query. In geopy 2.0 this will become an exception.
  'exception.' % str(e), UserWarning


In [ ]:
# further analysis explorative gives us more text analysis details
profile_2 = ProfileReport(user_profile_clean, title='User Profile Report', 
                          explorative=True)
profile_2

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# rename some columns

user_profile_clean.rename({'user.username': 'Instagram Handle',
                           'user.is_business' : 'Business (T/F)',
                           'user.category' : 'Business / Pro Type',
                           'user.full_name' : 'Business / Pro Name',
                           'user.city_name' :  'City',
                           'user.zip' : 'Zip / Postal',
                           'user.public_email': 'Email',
                           'user.public_phone_number': 'Phone Number 1',
                           'user.contact_phone_number': 'Phone Number 2',
                           'user.external_url' : 'Website',
                           'user.address_street' : 'Street Address'
                           },
                          axis='columns', inplace=True)
user_profile_clean

,input.user_id,Instagram Handle,user.biography,Website,Street Address,Business / Pro Name,Business / Pro Type,City,Phone Number 2,Email,user.public_phone_country_code,Phone Number 1,Zip / Postal,Business (T/F),user.is_potential_business,user.account_type
0,5932394085,tammycburrows,Saskatchewan based Hairstylist for 30+ years!\...,NaN,1142 Dorothy Street,Tammy W | yqr hairstylist,Health/Beauty,"Regina, Saskatchewan",13069242000.0,NaN,1.0,3.069242e+09,NaN,True,False,2.0
1,6898442222,mad.creationz,🤍hair student in vancouver-ish bc\n🤍personal: ...,NaN,NaN,madi baines,"Beauty, Cosmetic & Personal Care",NaN,NaN,madizmakeup@gmail.com,NaN,NaN,NaN,True,True,2.0
2,6905584184,sallyhawkins_schwarzkopf,Sales Manager- Schwarzkopf,http://www.skpevents.co.uk/virtualeduk8,NaN,Sally Hawkins-Schwarzkopf,Health/Beauty,NaN,447785401529.0,sally.hawkins@henkel.com,44.0,7.785402e+09,NaN,True,False,2.0
3,6924108892,ellanashairhive_,Welcome hair honeys to my Instagram🍯 • Junior...,NaN,NaN,Ellana,Beauty Salon,NaN,16134037640.0,e.bobbyb123@gmail.com,1.0,6.134038e+09,NaN,True,True,2.0
4,7064134243,envybeautysalonandspa,"Where there is beauty, there is Envy.",NaN,"5,6 - 2397, Finch Avenue West",Envy Beauty Salon & Spa,Health/Beauty,"Toronto, Ontario",14167405231.0,NaN,1.0,4.167405e+09,M9M 2X1,True,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,236853054,joyymia,𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...,NaN,NaN,𝐌̃𝐢̃𝐚̃ ☾,Scientist,NaN,NaN,NaN,NaN,NaN,NaN,False,True,3.0
3664,356998227,sweet_sb05,"“ It’s nice to be important , but it’s more im...",NaN,NaN,Sonia 🇱🇧 / 🇨🇦,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
3665,512109425,hirtho79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0
3666,554098691,like_alyssia,✨ accepting new clients ✨ \nBlondes&Balayage\n...,https://tinyurl.com/yd65fzta,NaN,Alyssia | Winnipeg MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,3.0


In [ ]:
# groupby username and collapse
#user_profile_clean_2 = user_profile_clean.groupby('Instagram Handle', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
user_profile_clean = user_profile_clean.groupby('user.username', as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))
user_profile_clean

,user.username,input.user_id,user.biography,user.external_url,user.address_street,user.full_name,user.category,user.city_name,user.contact_phone_number,user.public_email,user.public_phone_country_code,user.public_phone_number,user.zip,user.is_business,user.is_potential_business,user.account_type,user.latitude,user.longitude
0,01medrano,3523903075,La vida es un paseo 🌎 🛫￼\n🇻🇪 ￼🇵🇦 🇪🇨 🇵🇪 🇨🇱 ￼￼￼,nan,nan,Medranojose,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
1,02394k,3463019492,nan,nan,nan,sjskdo77,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
2,0_0_owl_eye_0_0,3616261218,nan,nan,nan,owl_eye,Personal Blog,"Baku, Azerbaijan",+994708224299,nan,994.0,708224299.0,nan,True,True,2.0,40.3953,49.8822
3,0_katinka_p_0,3518807757,nan,nan,nan,Katharina,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
4,123456ankb,8485698542,nan,nan,nan,ankb,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,zozan_guzellik_salonu,945391014,nan,nan,mevlana celalettin Rumi cad. başakşehir,Zozangokgüzelliksalonu,Health/Beauty,"Istanbul, Turkey",nan,nan,nan,nan,nan,True,True,2.0,41.09404,28.80843
2606,zullyalburjas,1135193946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
2607,zynpklc3434,3558315689,nan,nan,nan,zeynep Kiliç,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan
2608,zytka79,1063031928,nan,nan,nan,lucyna wojtanowicz,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan


In [ ]:
import re

# column with possible phone numbers extracted from the bio field
user_profile_clean['Phone Number 3'] = user_profile_clean['user.biography'].str.extract(
    "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")

# column with possible emails extracted from the bio field
user_profile_clean['Email 2'] = user_profile_clean['user.biography'].str.findall(
    "[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+")

user_profile_clean

,user.username,input.user_id,user.biography,user.external_url,user.address_street,user.full_name,user.category,user.city_name,user.contact_phone_number,user.public_email,user.public_phone_country_code,user.public_phone_number,user.zip,user.is_business,user.is_potential_business,user.account_type,user.latitude,user.longitude,Phone Number 3,Email 2
0,01medrano,3523903075,La vida es un paseo 🌎 🛫￼\n🇻🇪 ￼🇵🇦 🇪🇨 🇵🇪 🇨🇱 ￼￼￼,nan,nan,Medranojose,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
1,02394k,3463019492,nan,nan,nan,sjskdo77,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
2,0_0_owl_eye_0_0,3616261218,nan,nan,nan,owl_eye,Personal Blog,"Baku, Azerbaijan",+994708224299,nan,994.0,708224299.0,nan,True,True,2.0,40.3953,49.8822,NaN,[]
3,0_katinka_p_0,3518807757,nan,nan,nan,Katharina,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
4,123456ankb,8485698542,nan,nan,nan,ankb,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,zozan_guzellik_salonu,945391014,nan,nan,mevlana celalettin Rumi cad. başakşehir,Zozangokgüzelliksalonu,Health/Beauty,"Istanbul, Turkey",nan,nan,nan,nan,nan,True,True,2.0,41.09404,28.80843,NaN,[]
2606,zullyalburjas,1135193946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
2607,zynpklc3434,3558315689,nan,nan,nan,zeynep Kiliç,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]
2608,zytka79,1063031928,nan,nan,nan,lucyna wojtanowicz,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[]


In [ ]:
# keyword search on bio, we will try this using the RAKE library
!pip install -U nltk
!pip install rake-nltk


In [ ]:
 def rake_implement(x):
    from  rake_nltk import Rake 
    r = Rake()
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()

user_profile_clean['Keywords'] = user_profile_clean['user.biography'].apply(lambda x: rake_implement(str(x)))
user_profile_clean[['user.biography', 'Keywords']]

,user.biography,Keywords
0,Saskatchewan based Hairstylist for 30+ years!\...,[💈 barber ⁣ ✂ stylist ⁣ 🎨 colourist ⁣ ⁣ accept...
1,🤍hair student in vancouver-ish bc\n🤍personal: ...,"[ish bc 🤍 personal, 🤍 hair student, madiibaine..."
2,Sales Manager- Schwarzkopf,"[sales manager, schwarzkopf]"
3,Welcome hair honeys to my Instagram🍯 • Junior...,[instagram 🍯 • junior hairstylist • profession...
4,"Where there is beauty, there is Envy.","[envy, beauty]"
...,...,...
3663,𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...,[𝐖 ̃ 𝐢 ̃ 𝐭 ̃ 𝐜 ̃ 𝐡 ̃ 𝐲 ̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜...
3664,"“ It’s nice to be important , but it’s more im...","[nice “, “, nice, ’, important]"
3665,NaN,[nan]
3666,✨ accepting new clients ✨ \nBlondes&Balayage\n...,"[✨ accepting new clients ✨ blondes, balayage p..."


In [ ]:
# given keyword extraction
business_keywords = ["hair salon", "nail salon", "spa", "barbershop"]
pro_keywords = ["hair stylist", "nail technician", "esthetician", "Barber"]
services_keywords = ["women's haircut", "men's haircut", "hair color", "hair extensions", "balayage", "highlights"]


# get rid of delimeters? "(?=("+'|'.join(services_keywords)+r"))"
#user_profile_clean['Business Type'] = user_profile_clean['user.biography'].str.findall("(?i)(?=("+'|'.join(business_keywords)+r"))")
#user_profile_clean['Pro Type'] = user_profile_clean['user.biography'].str.findall("(?i)(?=("+'|'.join(pro_keywords)+r"))")
#user_profile_clean['Services'] = user_profile_clean['user.biography'].str.findall("(?i)(?=("+'|'.join(services_keywords)+r"))")

# new approach search all the fields
user_profile_clean['Business Type'] = user_profile_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(business_keywords)+"))") \
                                                          .fillna('')

user_profile_clean['Pro Type'] = user_profile_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(pro_keywords)+"))") \
                                                          .fillna('')

user_profile_clean['Services'] = user_profile_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(services_keywords)+"))") \
                                                          .fillna('')

'''                                                        
user_profile_clean['Business Type'] = user_profile_clean[['user.biography', 
                                                          'Instagram Handle', 
                                                          'Website', 
                                                          'Business / Pro Name', 
                                                          'Business / Pro Type', 
                                                          'Email']] \
                                                          .apply(lambda x: 
                                                                ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(business_keywords)+"))") \
                                                          .fillna('')

user_profile_clean['Pro Type'] = user_profile_clean[['user.biography', 
                                                          'Instagram Handle', 
                                                          'Website', 
                                                          'Business / Pro Name', 
                                                          'Business / Pro Type', 
                                                          'Email']] \
                                                          .apply(lambda x: 
                                                                ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(pro_keywords)+"))") \
                                                          .fillna('')

user_profile_clean['Services'] = user_profile_clean[['user.biography', 
                                                          'Instagram Handle', 
                                                          'Website', 
                                                          'Business / Pro Name', 
                                                          'Business / Pro Type', 
                                                          'Email']] \
                                                          .apply(lambda x: 
                                                                ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(services_keywords)+"))") \
                                                          .fillna('')
'''
user_profile_clean


,user.username,input.user_id,user.biography,user.external_url,user.address_street,user.full_name,user.category,user.city_name,user.contact_phone_number,user.public_email,user.public_phone_country_code,user.public_phone_number,user.zip,user.is_business,user.is_potential_business,user.account_type,user.latitude,user.longitude,Phone Number 3,Email 2,Business Type,Pro Type,Services
0,01medrano,3523903075,La vida es un paseo 🌎 🛫￼\n🇻🇪 ￼🇵🇦 🇪🇨 🇵🇪 🇨🇱 ￼￼￼,nan,nan,Medranojose,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
1,02394k,3463019492,nan,nan,nan,sjskdo77,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
2,0_0_owl_eye_0_0,3616261218,nan,nan,nan,owl_eye,Personal Blog,"Baku, Azerbaijan",+994708224299,nan,994.0,708224299.0,nan,True,True,2.0,40.3953,49.8822,NaN,[],[],[],[]
3,0_katinka_p_0,3518807757,nan,nan,nan,Katharina,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
4,123456ankb,8485698542,nan,nan,nan,ankb,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,zozan_guzellik_salonu,945391014,nan,nan,mevlana celalettin Rumi cad. başakşehir,Zozangokgüzelliksalonu,Health/Beauty,"Istanbul, Turkey",nan,nan,nan,nan,nan,True,True,2.0,41.09404,28.80843,NaN,[],[],[],[]
2606,zullyalburjas,1135193946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
2607,zynpklc3434,3558315689,nan,nan,nan,zeynep Kiliç,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]
2608,zytka79,1063031928,nan,nan,nan,lucyna wojtanowicz,nan,nan,nan,nan,nan,nan,nan,False,False,1.0,nan,nan,NaN,[],[],[],[]


In [ ]:
# now lets look at the user comments

# Instagram user posts data: 
user_posts = pd.read_csv('https://se-data-test.s3.ca-central-1.amazonaws.com/user_posts_v1.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,7,10,11,14,16,17,20,22,23,29,30,31,32,34,35,36,37,38,40,49,50,61,88,90,91,92,103,104,105,107,108,109,110,111,112,116,119,120,123,124,128,132,133,135,136,138,139,140,141,142,143,145,150,154,155,157,158,160,161,162,163,164,165,167,171,172,173,174,175,176,178,179,181,182,183,184,185,186,188,189,191,192,193,195,196,197,198,199,201,205,206,208,209,210,211,212,223,224,227,228,229,230,231,232,233,238,241,245,247,249) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
user_posts.head()

,input.session_id,input.username,input.max_id,items.taken_at,items.pk,items.id,items.device_timestamp,items.media_type,items.code,items.client_cache_key,items.filter_type,items.location.pk,items.location.name,items.location.address,items.location.city,items.location.short_name,items.location.lng,items.location.lat,items.location.external_source,items.location.facebook_places_id,items.lat,items.lng,items.should_request_ads,items.user.pk,items.user.username,items.user.full_name,items.user.is_private,items.user.profile_pic_url,items.user.profile_pic_id,items.user.is_verified,items.user.has_anonymous_profile_picture,items.user.is_unpublished,items.user.is_favorite,items.user.latest_reel_media,items.can_viewer_reshare,items.caption_is_edited,items.comment_likes_enabled,items.comment_threading_enabled,items.has_more_comments,items.max_num_visible_preview_comments,...,items.creative_config.effect_preview.effect_action_sheet.secondary_actions[0],items.creative_config.effect_preview.save_status,items.creative_config.effect_preview.device_position,items.media_cropping_info.square_crop.crop_left,items.media_cropping_info.square_crop.crop_right,items.media_cropping_info.square_crop.crop_top,items.media_cropping_info.square_crop.crop_bottom,items.media_cropping_info.feed_preview_crop.crop_left,items.media_cropping_info.feed_preview_crop.crop_right,items.media_cropping_info.feed_preview_crop.crop_top,items.media_cropping_info.feed_preview_crop.crop_bottom,items.igtv_series_info.id,items.igtv_series_info.title,items.igtv_series_info.has_cover_photo,items.igtv_series_info.num_episodes,items.sponsor_tags[0].pk,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name,items.sponsor_tags[0].is_private,items.sponsor_tags[0].profile_pic_url,items.sponsor_tags[0].profile_pic_id,items.sponsor_tags[0].is_verified,items.sponsor_tags[0].permission,items.location.facebook_events_id,items.location.start_time,items.location.end_time,items.location.time_granularity,items.location.timezone,items.location.created_at,items.location.event_category,items.location.place_name,items.location.place_fbid,items.location.country,items.sponsor_tags[0],items.commenting_disabled_for_viewer,num_results,more_available,next_max_id,auto_load_more_enabled,status
0,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,NaN,1.602424e+09,2417632317842615770,2417632317842615770_16472886710,1.602410e+15,1,CGNJ8iug23a,MjQxNzYzMjMxNzg0MjYxNTc3MA==.2,0,1.41284e+15,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,-3.03222,53.8263,facebook_places,1.412839e+15,53.8263,-3.032216,False,16472886710,hairbypetrina,Hair By Petrina,False,https://scontent.cdninstagram.com/v/t51.2885-1...,2262739819517992795_16472886710,False,False,False,False,0.0,True,False,True,False,True,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,True,2363183485612567305_16472886710,True,ok
1,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,NaN,1.601142e+09,2406878064765178989,2406878064765178989_16472886710,1.601142e+15,1,CFm8tkEAhRt,MjQwNjg3ODA2NDc2NTE3ODk4OQ==.2,0,1.41284e+15,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,-3.03222,53.8263,facebook_places,1.412839e+15,53.8263,-3.032216,False,16472886710,hairbypetrina,Hair By Petrina,False,https://scontent.cdninstagram.com/v/t51.2885-1...,2262739819517992795_16472886710,False,False,False,False,0.0,True,False,True,False,True,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,True,2363183485612567305_16472886710,True,ok
2,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,NaN,1.601142e+09,2406873404524213624,2406873404524213624_16472886710,1.601141e+15,1,CFm7pv4ADV4,MjQwNjg3MzQwNDUyNDIxMzYyNA==.2,0,1.41284e+15,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,-3.03222,53.8263,facebook_pla

In [ ]:
profile_posts = ProfileReport(user_posts, minimal=True)
profile_posts

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# grab some of the relevant columns
user_posts_clean = user_posts[['input.session_id',
                               'input.username',
                               'items.location.name',
                               'items.location.address',
                               'items.location.city',
                               'items.location.short_name',
                               'items.user.username',
                               'items.user.full_name',
                               'items.usertags.in[0].user.username',
                               'items.usertags.in[0].user.full_name',
                               'items.caption.text',
                               'items.caption.user.full_name',
                               'items.caption.user.username',
                               'items.preview_comments[0].text',
                               'items.preview_comments[0].user.username',
                               'items.preview_comments[0].user.full_name',
                               'items.comments[0].text',
                               'items.comments[0].user.username',
                               'items.comments[0].user.full_name',
                               'items.likers[0].username',
                               'items.likers[0].full_name',
                               'items.creative_config.effect_preview.name',
                               'items.creative_config.effect_preview.attribution_user.username',
                               'items.creative_config.effect_preview.gatekeeper',
                               'items.igtv_series_info.title',
                               'items.sponsor_tags[0].username',
                               'items.sponsor_tags[0].full_name'
                               ]].copy()
user_posts_clean

,input.session_id,input.username,items.location.name,items.location.address,items.location.city,items.location.short_name,items.user.username,items.user.full_name,items.usertags.in[0].user.username,items.usertags.in[0].user.full_name,items.caption.text,items.caption.user.full_name,items.caption.user.username,items.preview_comments[0].text,items.preview_comments[0].user.username,items.preview_comments[0].user.full_name,items.comments[0].text,items.comments[0].user.username,items.comments[0].user.full_name,items.likers[0].username,items.likers[0].full_name,items.creative_config.effect_preview.name,items.creative_config.effect_preview.attribution_user.username,items.creative_config.effect_preview.gatekeeper,items.igtv_series_info.title,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name
0,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,hairbypetrina,Hair By Petrina,sexyhair,SexyHair®,𝙰𝚋𝚜𝚘𝚞𝚕𝚝𝚎 - 𝚑𝚊𝚒𝚛 - 𝚝𝚛𝚊𝚗𝚜𝚏𝚘𝚛𝚖𝚝𝚒𝚘𝚗 \n\n𝚂𝚎𝚛𝚟𝚒𝚌𝚎: B...,Hair By Petrina,hairbypetrina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,hairbypetrina,Hair By Petrina,sexyhair,SexyHair®,Ａｂｓｏｌｕｔｅ － Ｂａｌａｙａｇｅ － ｇｏａｌｓ \n\nＨａｉｒ － ｂｙ － ｐｅ...,Hair By Petrina,hairbypetrina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,hairbypetrina,Hair By Petrina,sexyhair,SexyHair®,Ｔｒａｎｓｆｏｒｍａｔｉｏｎ － ｏｆ － ｔｈｅ － ｄａｙ \n\nＨａｉｒ -ｂｙ -...,Hair By Petrina,hairbypetrina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,Becky Janade Hair & Beauty,41 Layton road,Blackpool,Becky Janade Hair & Beauty,hairbypetrina,Hair By Petrina,beckyjanadehairandbeauty,Becky Janade Hair & Beauty Ltd,Had the absolute pleasure to cut and colour th...,Hair By Petrina,hairbypetrina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43098382060%3AWm6UHSrKX6W3gW%3A1,hairbypetrina,Amaro Restaurant,109 Church Street,Blackpool,Amaro Restaurant,hairbypetrina,Hair By Petrina,NaN,NaN,𝑯𝒂𝒅 𝒕𝒉𝒆 𝒃𝒆𝒔𝒕 𝒏𝒊𝒈𝒉𝒕 𝒘𝒊𝒕𝒉 𝒕𝒉𝒆 𝒃𝒆𝒔𝒕 𝒈𝒊𝒓𝒍𝒔 \n\n𝑭𝒐𝒐...,Hair By Petrina,hairbypetrina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549656,43098382060%3AWm6UHSrKX6W3gW%3A1,haircandymd,Hair Candy LLC,"113 Baptist St, Ste 2","Salisbury, Maryland",Hair Candy LLC,haircandymd,💗Hair Candy LLC💗🌈✂️🍭🎨,NaN,NaN,Lots of regrowth but we did a bunch of babylig...,💗Hair Candy LLC💗🌈✂️🍭🎨,haircandymd,NaN,NaN,NaN,NaN,NaN,NaN,leesawebster,Leesa Webster,NaN,NaN,NaN,NaN,NaN,NaN
549657,43098382060%3AWm6UHSrKX6W3gW%3A1,haircandymd,NaN,NaN,NaN,NaN,haircandymd,💗Hair Candy LLC💗🌈✂️🍭🎨,abbybueck,abbybueck,Look how pretty and dimensional this is! Had t...,💗Hair Candy LLC💗🌈✂️🍭🎨,haircandymd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549658,43098382060%3AWm6UHSrKX6W3gW%3A1,haircandymd,Hair Candy LLC,"113 Baptist St, Ste 2","Salisbury, Maryland",Hair Candy LLC,haircandymd,💗Hair Candy LLC💗🌈✂️🍭🎨,NaN,NaN,5.5 hour transformation! 🌈🌈🌈🌈,💗Hair Candy LLC💗🌈✂️🍭🎨,haircandymd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549659,43098382060%3AWm6UHSrKX6W3gW%3A1,haircandymd,Hair Candy LLC,"113 Baptist St, Ste 2","Salisbury, Maryland",Hair Candy LLC,haircandymd,💗Hair Candy LLC💗🌈✂️🍭🎨,NaN,NaN,Mermaids are real thou🧜‍♀️🧜‍♀️ #amirite #merma...,💗Hair Candy LLC💗🌈✂️🍭🎨,haircandymd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# groupby username and collapse
user_posts_clean = user_posts_clean.groupby('input.username', as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))
user_posts_clean

,input.username,input.session_id,items.location.name,items.location.address,items.location.city,items.location.short_name,items.user.username,items.user.full_name,items.usertags.in[0].user.username,items.usertags.in[0].user.full_name,items.caption.text,items.caption.user.full_name,items.caption.user.username,items.preview_comments[0].text,items.preview_comments[0].user.username,items.preview_comments[0].user.full_name,items.comments[0].text,items.comments[0].user.username,items.comments[0].user.full_name,items.likers[0].username,items.likers[0].full_name,items.creative_config.effect_preview.name,items.creative_config.effect_preview.attribution_user.username,items.creative_config.effect_preview.gatekeeper,items.igtv_series_info.title,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name
0,1.0,"#Hers, #decor #job #work #hairdresser's #welco...",False,https://scontent.cdninstagram.com/v/t51.2885-1...,"nan, 1464774226530592861_319967420, 1863077793...",False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,hairbypetrina,43098382060%3AWm6UHSrKX6W3gW%3A1,"Becky Janade Hair & Beauty, nan, Amaro Restaurant","nan, 109 Church Street, 41 Layton road","nan, Blackpool","Becky Janade Hair & Beauty, nan, Amaro Restaurant",hairbypetrina,Hair By Petrina,"nan, shelbi_lorrenna, kaideesianx, mollybeal_,...","Revlon, YASMIN HESKETH, nan, Neal and Wolf, Se...",Ｂｌｏｎｄｅ － ａｌｅｒｔ 🚨. Ａｂｓｏｌｕｔｅ － ｈａｉｒ -ｔｒａｎｓｆｏｒｍａ...,Hair By Petrina,hairbypetrina,"nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, charlie_priestleyxo, bigbang_vpc, pelval5...","nan, Big Bang Vita Per Capelli, Princess Jas, ...",nan,nan,nan,nan,nan,nan
2,hairbypeytonxx,43098382060%3AWm6UHSrKX6W3gW%3A1,"The Davison Hotel, Intuitive Touch, Windsor Pl...","6029 Lapeer Rd, 7410 E. COURT STREET, 105 E. S...","Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","The Davison Hotel, Flint, Intuitive Touch, Win...",hairbypeytonxx,Lovely Locks By Peyton,"nigglywiggly, katie.akins, lexie_harburn, elen...","Madison Alexander, Austin Eaton, Sadie, Sarah ...",I will be opening up June 15th! Thank you for ...,"Lovely Locks By Peyton, nan","nan, hairbypeytonxx",nan,nan,nan,nan,nan,nan,"terrincarlson, nigglywiggly, gmoneynels, hanna...","Paige Vance🌻, E, Kayla Sorenson, Michael Dueñ...",nan,nan,nan,nan,nan,nan
3,hairbypinkyd,43098382060%3AWm6UHSrKX6W3gW%3A1,"Atlantic Beach, Florida, Jacksonville Beach, F...","331 Marsh Landing Pkwy, nan, 200 Ponte Vedra B...","nan, Jacksonville Beach, Florida, Ponte Vedra ...","Shear Artistry By Torrie, New York, Paul Mitch...",hairbypinkyd,Pinky Denton,"lorealpro, adri_316, pulpriothair, veronichhhk...","Mollie, American Salon, Keratin Complex, Ted G...",Happy Sunday!!! Please visit my business page ...,"Pinky Denton, nan","hairbypinkyd, nan","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","jennyleepizzo, hannahruth.g, jbheurope, smashe...","⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Tavi 🌙💗🌟, jennyleepizzo, Mollie, Alw...",nan,nan,nan,nan,nan,nan
4,hairbypizer,43098382060%3AWm6UHSrKX6W3gW%3A1,"Boston Logan International Airport, Glenstone ...","1600 7th St NW, 4155 Linnean Ave NW, 1614 Wisc...","Washington, nan, Nashville, TN, Fairfax County...","Boston Logan International Airport, Glenstone ...",hairbypizer,Madison Pizer | DC Stylist,"omgartistry, sophiemarcus, bescene, pmthetempl...","Angie Greer, Warby Parker, Jordan Swarm, Olivi...","Inner Conflict: loves snow, hates cold. Summer...","nan, Madison Pizer | DC Stylist","hairbypizer, nan","nan, 👏👏👏","amika, nan","amika, nan","nan, 👏👏👏","amika, nan","amika, nan",nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,store6546,43714256488%3A1blswu8AeB4C7m%3A14,"nan, Cosmoprof","nan, 2185 Cunningham Dr",nan,"nan, Cosmoprof",store654

In [ ]:
# search all the fields
user_posts_clean['Business Type'] = user_posts_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(business_keywords)+"))") \
                                                          .fillna('')

user_posts_clean['Pro Type'] = user_posts_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(pro_keywords)+"))") \
                                                          .fillna('')

user_posts_clean['Services'] = user_posts_clean.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1) \
                                                          .str.findall("(?i)(?=("+'|'.join(services_keywords)+"))") \
                                                          .fillna('')


user_posts_clean

,input.username,input.session_id,items.location.name,items.location.address,items.location.city,items.location.short_name,items.user.username,items.user.full_name,items.usertags.in[0].user.username,items.usertags.in[0].user.full_name,items.caption.text,items.caption.user.full_name,items.caption.user.username,items.preview_comments[0].text,items.preview_comments[0].user.username,items.preview_comments[0].user.full_name,items.comments[0].text,items.comments[0].user.username,items.comments[0].user.full_name,items.likers[0].username,items.likers[0].full_name,items.creative_config.effect_preview.name,items.creative_config.effect_preview.attribution_user.username,items.creative_config.effect_preview.gatekeeper,items.igtv_series_info.title,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name,Business Type,Pro Type,Services
0,1.0,"#Hers, #decor #job #work #hairdresser's #welco...",False,https://scontent.cdninstagram.com/v/t51.2885-1...,"nan, 1464774226530592861_319967420, 1863077793...",False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,[],[],[balayage]
1,hairbypetrina,43098382060%3AWm6UHSrKX6W3gW%3A1,"Becky Janade Hair & Beauty, nan, Amaro Restaurant","nan, 109 Church Street, 41 Layton road","nan, Blackpool","Becky Janade Hair & Beauty, nan, Amaro Restaurant",hairbypetrina,Hair By Petrina,"nan, shelbi_lorrenna, kaideesianx, mollybeal_,...","Revlon, YASMIN HESKETH, nan, Neal and Wolf, Se...",Ｂｌｏｎｄｅ － ａｌｅｒｔ 🚨. Ａｂｓｏｌｕｔｅ － ｈａｉｒ -ｔｒａｎｓｆｏｒｍａ...,Hair By Petrina,hairbypetrina,"nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, charlie_priestleyxo, bigbang_vpc, pelval5...","nan, Big Bang Vita Per Capelli, Princess Jas, ...",nan,nan,nan,nan,nan,nan,[],[],"[balayage, balayage, balayage, balayage, balay..."
2,hairbypeytonxx,43098382060%3AWm6UHSrKX6W3gW%3A1,"The Davison Hotel, Intuitive Touch, Windsor Pl...","6029 Lapeer Rd, 7410 E. COURT STREET, 105 E. S...","Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","The Davison Hotel, Flint, Intuitive Touch, Win...",hairbypeytonxx,Lovely Locks By Peyton,"nigglywiggly, katie.akins, lexie_harburn, elen...","Madison Alexander, Austin Eaton, Sadie, Sarah ...",I will be opening up June 15th! Thank you for ...,"Lovely Locks By Peyton, nan","nan, hairbypeytonxx",nan,nan,nan,nan,nan,nan,"terrincarlson, nigglywiggly, gmoneynels, hanna...","Paige Vance🌻, E, Kayla Sorenson, Michael Dueñ...",nan,nan,nan,nan,nan,nan,[spa],"[barber, hair stylist, barber]","[hair color, balayage, balayage, BALAYAGE, hig..."
3,hairbypinkyd,43098382060%3AWm6UHSrKX6W3gW%3A1,"Atlantic Beach, Florida, Jacksonville Beach, F...","331 Marsh Landing Pkwy, nan, 200 Ponte Vedra B...","nan, Jacksonville Beach, Florida, Ponte Vedra ...","Shear Artistry By Torrie, New York, Paul Mitch...",hairbypinkyd,Pinky Denton,"lorealpro, adri_316, pulpriothair, veronichhhk...","Mollie, American Salon, Keratin Complex, Ted G...",Happy Sunday!!! Please visit my business page ...,"Pinky Denton, nan","hairbypinkyd, nan","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","jennyleepizzo, hannahruth.g, jbheurope, smashe...","⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Tavi 🌙💗🌟, jennyleepizzo, Mollie, Alw...",nan,nan,nan,nan,nan,nan,"[spa, Hair Salon, spa]",[],"[Hair Color, highlights, balayage, balayage, b..."
4,hairbypizer,43098382060%3AWm6UHSrKX6W3gW%3A1,"Boston Logan International Airport, Glenstone ...","1600 7th St NW, 4155 Linnean Ave NW, 1614 Wisc...","Washington, nan, Nashville, TN, Fairfax County...","Boston Logan International Airport, Glenstone ...",hairbypizer,Madison Pizer | DC Stylist,"omgartistry, sophiemarcus, bescene, pmthetempl...","Angie Greer, Warby Parker, Jordan Swarm, Olivi...","Inner Conflict: loves snow, hates cold. Summer...","nan, Madison Pizer | DC Stylist","hairbypizer, nan","nan, 👏👏👏","amika, nan","amika, nan","nan, 👏👏👏","amika, nan",

In [ ]:
user_profile_clean['user.username'].isin(user_posts_clean['input.username']).value_counts()

False    2576
True       34
Name: user.username, dtype: int64

In [ ]:
users_merged = pd.merge(user_posts_clean, user_profile_clean, how='outer', left_on=['input.username'], right_on=['user.username'])
users_merged

,input.username,input.session_id,items.location.name,items.location.address,items.location.city,items.location.short_name,items.user.username,items.user.full_name,items.usertags.in[0].user.username,items.usertags.in[0].user.full_name,items.caption.text,items.caption.user.full_name,items.caption.user.username,items.preview_comments[0].text,items.preview_comments[0].user.username,items.preview_comments[0].user.full_name,items.comments[0].text,items.comments[0].user.username,items.comments[0].user.full_name,items.likers[0].username,items.likers[0].full_name,items.creative_config.effect_preview.name,items.creative_config.effect_preview.attribution_user.username,items.creative_config.effect_preview.gatekeeper,items.igtv_series_info.title,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name,Business Type_x,Pro Type_x,Services_x,user.username,input.user_id,user.biography,user.external_url,user.address_street,user.full_name,user.category,user.city_name,user.contact_phone_number,user.public_email,user.public_phone_country_code,user.public_phone_number,user.zip,user.is_business,user.is_potential_business,user.account_type,user.latitude,user.longitude,Phone Number 3,Email 2,Business Type_y,Pro Type_y,Services_y
0,1.0,"#Hers, #decor #job #work #hairdresser's #welco...",False,https://scontent.cdninstagram.com/v/t51.2885-1...,"nan, 1464774226530592861_319967420, 1863077793...",False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,[],[],[balayage],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hairbypetrina,43098382060%3AWm6UHSrKX6W3gW%3A1,"Becky Janade Hair & Beauty, nan, Amaro Restaurant","nan, 109 Church Street, 41 Layton road","nan, Blackpool","Becky Janade Hair & Beauty, nan, Amaro Restaurant",hairbypetrina,Hair By Petrina,"nan, shelbi_lorrenna, kaideesianx, mollybeal_,...","Revlon, YASMIN HESKETH, nan, Neal and Wolf, Se...",Ｂｌｏｎｄｅ － ａｌｅｒｔ 🚨. Ａｂｓｏｌｕｔｅ － ｈａｉｒ -ｔｒａｎｓｆｏｒｍａ...,Hair By Petrina,hairbypetrina,"nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, charlie_priestleyxo, bigbang_vpc, pelval5...","nan, Big Bang Vita Per Capelli, Princess Jas, ...",nan,nan,nan,nan,nan,nan,[],[],"[balayage, balayage, balayage, balayage, balay...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hairbypeytonxx,43098382060%3AWm6UHSrKX6W3gW%3A1,"The Davison Hotel, Intuitive Touch, Windsor Pl...","6029 Lapeer Rd, 7410 E. COURT STREET, 105 E. S...","Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","The Davison Hotel, Flint, Intuitive Touch, Win...",hairbypeytonxx,Lovely Locks By Peyton,"nigglywiggly, katie.akins, lexie_harburn, elen...","Madison Alexander, Austin Eaton, Sadie, Sarah ...",I will be opening up June 15th! Thank you for ...,"Lovely Locks By Peyton, nan","nan, hairbypeytonxx",nan,nan,nan,nan,nan,nan,"terrincarlson, nigglywiggly, gmoneynels, hanna...","Paige Vance🌻, E, Kayla Sorenson, Michael Dueñ...",nan,nan,nan,nan,nan,nan,[spa],"[barber, hair stylist, barber]","[hair color, balayage, balayage, BALAYAGE, hig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hairbypinkyd,43098382060%3AWm6UHSrKX6W3gW%3A1,"Atlantic Beach, Florida, Jacksonville Beach, F...","331 Marsh Landing Pkwy, nan, 200 Ponte Vedra B...","nan, Jacksonville Beach, Florida, Ponte Vedra ...","Shear Artistry By Torrie, New York, Paul Mitch...",hairbypinkyd,Pinky Denton,"lorealpro, adri_316, pulpriothair, veronichhhk...","Mollie, American Salon, Keratin Complex, Ted G...",Happy Sunday!!! Please visit my business page ...,"Pinky Denton, nan","hairbypinkyd, nan","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","nan, 💕","nan, lilmamikara","nan, kara ubriaco~","jennyleepizzo, hannahruth.g, jbheurope, smashe...","⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Tavi 🌙💗🌟, jennyleepizzo, Mollie, Alw...",

In [ ]:
users_merged["Instagram Handle"] = users_merged["input.username"].fillna(users_merged["user.username"])
users_merged["Business Type"] = users_merged["Business Type_x"].fillna(users_merged["Business Type_y"])
users_merged["Pro Type"] = users_merged["Pro Type_x"].fillna(users_merged["Pro Type_y"])
users_merged["Business Name"] = users_merged["user.full_name"].fillna(users_merged["items.user.full_name"])
users_merged["Pro Name"] = users_merged["Business Name"]
users_merged["City"] = users_merged["user.city_name"].fillna(users_merged["items.location.city"])
users_merged["Services"] = users_merged["Services_x"].fillna(users_merged["Services_y"])

users_merged.rename({'user.is_business' : 'Business (T/F)',
                     'user.zip' : 'Zip / Postal',
                     'user.public_email': 'Email 1',
                     'user.public_phone_number': 'Phone Number 1',
                     'user.contact_phone_number': 'Phone Number 2',
                     'user.external_url' : 'Website',
                     'user.address_street' : 'Street Address',
                     'user.latitude' : 'Latitude',
                     'user.longitude' : 'Longitude'
                    },
                  axis='columns', inplace=True)

users_merged
#users_merged.fillna('', inplace=True)
# name?? users_merged, items.user.full_name

,input.username,input.session_id,items.location.name,items.location.address,items.location.city,items.location.short_name,items.user.username,items.user.full_name,items.usertags.in[0].user.username,items.usertags.in[0].user.full_name,items.caption.text,items.caption.user.full_name,items.caption.user.username,items.preview_comments[0].text,items.preview_comments[0].user.username,items.preview_comments[0].user.full_name,items.comments[0].text,items.comments[0].user.username,items.comments[0].user.full_name,items.likers[0].username,items.likers[0].full_name,items.creative_config.effect_preview.name,items.creative_config.effect_preview.attribution_user.username,items.creative_config.effect_preview.gatekeeper,items.igtv_series_info.title,items.sponsor_tags[0].username,items.sponsor_tags[0].full_name,Business Type_x,Pro Type_x,Services_x,user.username,input.user_id,user.biography,Website,Street Address,user.full_name,user.category,user.city_name,Phone Number 2,Email 1,user.public_phone_country_code,Phone Number 1,Zip / Postal,Business (T/F),user.is_potential_business,user.account_type,Latitude,Longitude,Phone Number 3,Email 2,Business Type_y,Pro Type_y,Services_y,Instagram Handle,Business Type,Pro Type,Business Name,Pro Name,City,Services
0,1.0,"#Hers, #decor #job #work #hairdresser's #welco...",False,https://scontent.cdninstagram.com/v/t51.2885-1...,"nan, 1464774226530592861_319967420, 1863077793...",False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,[],[],[balayage],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,[],[],nan,nan,"nan, 1464774226530592861_319967420, 1863077793...",[balayage]
1,hairbypetrina,43098382060%3AWm6UHSrKX6W3gW%3A1,"Becky Janade Hair & Beauty, nan, Amaro Restaurant","nan, 109 Church Street, 41 Layton road","nan, Blackpool","Becky Janade Hair & Beauty, nan, Amaro Restaurant",hairbypetrina,Hair By Petrina,"nan, shelbi_lorrenna, kaideesianx, mollybeal_,...","Revlon, YASMIN HESKETH, nan, Neal and Wolf, Se...",Ｂｌｏｎｄｅ － ａｌｅｒｔ 🚨. Ａｂｓｏｌｕｔｅ － ｈａｉｒ -ｔｒａｎｓｆｏｒｍａ...,Hair By Petrina,hairbypetrina,"nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, Love 💖, Fab 🌟, Wow 🤩, So fab 🌟, 😍, 😍😍😍, B...","nan, salonsdirect","Salons Direct, nan","nan, charlie_priestleyxo, bigbang_vpc, pelval5...","nan, Big Bang Vita Per Capelli, Princess Jas, ...",nan,nan,nan,nan,nan,nan,[],[],"[balayage, balayage, balayage, balayage, balay...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hairbypetrina,[],[],Hair By Petrina,Hair By Petrina,"nan, Blackpool","[balayage, balayage, balayage, balayage, balay..."
2,hairbypeytonxx,43098382060%3AWm6UHSrKX6W3gW%3A1,"The Davison Hotel, Intuitive Touch, Windsor Pl...","6029 Lapeer Rd, 7410 E. COURT STREET, 105 E. S...","Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","The Davison Hotel, Flint, Intuitive Touch, Win...",hairbypeytonxx,Lovely Locks By Peyton,"nigglywiggly, katie.akins, lexie_harburn, elen...","Madison Alexander, Austin Eaton, Sadie, Sarah ...",I will be opening up June 15th! Thank you for ...,"Lovely Locks By Peyton, nan","nan, hairbypeytonxx",nan,nan,nan,nan,nan,nan,"terrincarlson, nigglywiggly, gmoneynels, hanna...","Paige Vance🌻, E, Kayla Sorenson, Michael Dueñ...",nan,nan,nan,nan,nan,nan,[spa],"[barber, hair stylist, barber]","[hair color, balayage, balayage, BALAYAGE, hig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hairbypeytonxx,[spa],"[barber, hair stylist, barber]",Lovely Locks By Peyton,Lovely Locks By Peyton,"Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","[hair color, balayage, balayage, BALAYAGE, hig..."
3,hairbypinkyd,43098382060%3AWm6UHSrKX6W3gW%3A1,"Atlantic Beach, Florida, Jacksonville Beach, F...","331 Marsh Landing Pkwy, nan, 200 Ponte Vedra B...","nan, Jacksonville Beach, Florida, Ponte Vedra ...","Shear Artistry By Torrie, New York, Paul Mit

In [ ]:
final = users_merged[['Instagram Handle',
                      'Business Type',
                      'Pro Type',
                      'Business Name',
                      'Pro Name',
                      'City',
                      'Services',
                      'Business (T/F)',
                      'Zip / Postal',
                      'Email 1',
                      'Email 2',
                      'Phone Number 1',
                      'Phone Number 2',
                      'Phone Number 3',
                      'Website',
                      'Street Address',
                      'Latitude',
                      'Longitude']].copy()

final['Phone Number'] = final[["Phone Number 1", "Phone Number 3", "Phone Number 3"]].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)

final['Email'] = final[["Email 1", "Email 2"]].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)

final.drop(['Phone Number 1', 'Phone Number 2', 'Phone Number 3', "Email 1", "Email 2"], axis=1, inplace=True)
final

,Instagram Handle,Business Type,Pro Type,Business Name,Pro Name,City,Services,Business (T/F),Zip / Postal,Website,Street Address,Latitude,Longitude,Phone Number,Email
0,1.0,[],[],nan,nan,"nan, 1464774226530592861_319967420, 1863077793...",[balayage],NaN,NaN,NaN,NaN,NaN,NaN,",,",","
1,hairbypetrina,[],[],Hair By Petrina,Hair By Petrina,"nan, Blackpool","[balayage, balayage, balayage, balayage, balay...",NaN,NaN,NaN,NaN,NaN,NaN,",,",","
2,hairbypeytonxx,[spa],"[barber, hair stylist, barber]",Lovely Locks By Peyton,Lovely Locks By Peyton,"Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","[hair color, balayage, balayage, BALAYAGE, hig...",NaN,NaN,NaN,NaN,NaN,NaN,",,",","
3,hairbypinkyd,"[spa, Hair Salon, spa]",[],Pinky Denton,Pinky Denton,"nan, Jacksonville Beach, Florida, Ponte Vedra ...","[Hair Color, highlights, balayage, balayage, b...",NaN,NaN,NaN,NaN,NaN,NaN,",,",","
4,hairbypizer,"[Spa, Spa, Hair Salon, spa, spa, spa, spa, spa...",[],Madison Pizer | DC Stylist,Madison Pizer | DC Stylist,"Washington, nan, Nashville, TN, Fairfax County...","[balayage, balayage, balayage, balayage, balay...",NaN,NaN,NaN,NaN,NaN,NaN,",,",","
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,zozan_guzellik_salonu,[],[],Zozangokgüzelliksalonu,Zozangokgüzelliksalonu,"Istanbul, Turkey",[],True,nan,nan,mevlana celalettin Rumi cad. başakşehir,41.09404,28.80843,"nan,,","nan,[]"
3742,zullyalburjas,[],[],nan,nan,nan,[],False,nan,nan,nan,nan,nan,"nan,,","nan,[]"
3743,zynpklc3434,[],[],zeynep Kiliç,zeynep Kiliç,nan,[],False,nan,nan,nan,nan,nan,"nan,,","nan,[]"
3744,zytka79,[],[],lucyna wojtanowicz,lucyna wojtanowicz,nan,[],False,nan,nan,nan,nan,nan,"nan,,","nan,[]"


In [ ]:
final.fillna('', inplace=True)
final

# remove duplicates
final['Services'] = final['Services'].apply(lambda x: str(set(x)))
final['Business Type'] = final['Business Type'].apply(lambda x: str(set(x)))
final['Pro Type'] = final['Pro Type'].apply(lambda x: str(set(x)))
#final['Email'] = final['Email'].apply(lambda x: set(x))
#final['Phone Number'] = final['Phone Number'].apply(lambda x: set(x))
final


,Instagram Handle,Business Type,Pro Type,Business Name,Pro Name,City,Services,Business (T/F),Zip / Postal,Website,Street Address,Latitude,Longitude,Phone Number,Email
0,1.0,set(),set(),nan,nan,"nan, 1464774226530592861_319967420, 1863077793...",{'balayage'},,,,,,,",,",","
1,hairbypetrina,set(),set(),Hair By Petrina,Hair By Petrina,"nan, Blackpool","{'balayage', 'highlights'}",,,,,,,",,",","
2,hairbypeytonxx,{'spa'},"{'barber', 'hair stylist'}",Lovely Locks By Peyton,Lovely Locks By Peyton,"Burton, MI, nan, Burton, Michigan, Lapeer, Cla...","{'BALAYAGE', 'Highlights', 'Balayage', 'balaya...",,,,,,,",,",","
3,hairbypinkyd,"{'Hair Salon', 'spa'}",set(),Pinky Denton,Pinky Denton,"nan, Jacksonville Beach, Florida, Ponte Vedra ...","{'Highlights', 'balayage', ""men's haircut"", 'h...",,,,,,,",,",","
4,hairbypizer,"{'Spa', 'Hair Salon', 'spa'}",set(),Madison Pizer | DC Stylist,Madison Pizer | DC Stylist,"Washington, nan, Nashville, TN, Fairfax County...","{'BALAYAGE', 'Balayage', 'balayage', 'highligh...",,,,,,,",,",","
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,zozan_guzellik_salonu,set(),set(),Zozangokgüzelliksalonu,Zozangokgüzelliksalonu,"Istanbul, Turkey",set(),True,nan,nan,mevlana celalettin Rumi cad. başakşehir,41.09404,28.80843,"nan,,","nan,[]"
3742,zullyalburjas,set(),set(),nan,nan,nan,set(),False,nan,nan,nan,nan,nan,"nan,,","nan,[]"
3743,zynpklc3434,set(),set(),zeynep Kiliç,zeynep Kiliç,nan,set(),False,nan,nan,nan,nan,nan,"nan,,","nan,[]"
3744,zytka79,set(),set(),lucyna wojtanowicz,lucyna wojtanowicz,nan,set(),False,nan,nan,nan,nan,nan,"nan,,","nan,[]"


In [ ]:
from pandas_profiling import ProfileReport

profile_fin = ProfileReport(final)
profile_fin